<a href="https://colab.research.google.com/github/juancarlosch1708-spec/IA/blob/main/CNN/Clasificacion_de_digitosJC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.datasets import load_digits

digits = load_digits()


Importa un conjunto de datos de dígitos escritos a mano (0–9) de scikit-learn y lo carga en la variable digits.
Este dataset se usa para entrenar y probar modelos de reconocimiento de números.

In [ ]:
digits.keys()


dict_keys(['data', 'target', 'frame', 'feature_names', 'target_names', 'images', 'DESCR'])

Muestra los nombres de los elementos que contiene el dataset digits (como imágenes, etiquetas, descripción, etc.).

In [ ]:
dict_keys([
    'data', 'target', 'frame',
    'feature_names', 'target_names', 'images',
    'DESCR'
])


Estos son los componentes del dataset digits:


data → Los datos de cada imagen en forma de números.


target → Las etiquetas (número real 0–9).


frame → (A veces vacío) versión tipo DataFrame.


feature_names → Nombres de características (no siempre usado).


target_names → Nombres de las clases (0–9).


images → Las imágenes en forma de matriz 8×8.


DESCR → Descripción del dataset.



In [ ]:
print(digits.DESCR)


Muestra la descripción completa del dataset digits:
incluye su origen, para qué sirve, cuántas muestras tiene y cómo están representadas las imágenes.

In [ ]:
index = 0


Guarda el número 0 en la variable index.
Esto normalmente se usa para seleccionar la primera imagen del dataset.

In [ ]:
image = digits.images[index]
label = digits.target[index]

digits.images contiene las imágenes del dataset digits (cada una es una matriz 8x8).

digits.target contiene las etiquetas correctas (el número que representa la imagen).

index indica cuál imagen escoger (como lo definiste, index = 0, toma la primera).

Entonces:

image ahora guarda la primera imagen (un dígito escrito a mano).

label guarda el número real representado en esa imagen.

In [ ]:
print(image)


imprime la imagen

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(3, 3))
plt.imshow(image, cmap=plt.cm.gray)
plt.title(f'Dígito: {label}')
plt.show()


esto mostrará el número que representa esa imagen.

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

X_train, X_test, y_train, y_test = train_test_split(
    digits.data, digits.target,
    test_size=0.2, random_state=42
)

# One-hot encode the target variable
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)


Se importa TensorFlow y herramientas para dividir datos.

In [ ]:
print(X_train.shape)
print(X_test.shape)


muestra el tamaño de los datos:

X_train.shape → cuántas imágenes se usan para entrenar.

X_test.shape → cuántas imágenes se usan para probar.

Cada imagen tiene 64 valores (porque es de 8x8 pixeles).

In [ ]:
reshaped_tensor = tf.reshape(X_train[0], shape=(8, 8))
print(reshaped_tensor)


está reorganizando el vector de 64 valores de la imagen (8×8 pixeles) nuevamente a su forma original de matriz 8x8, para poder verla como una imagen en lugar de una lista. Luego se imprime esa matriz.

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Normalizar los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

StandardScaler() ajusta los valores para que tengan media 0 y desviación estándar 1.
Esto ayuda a que la red neuronal aprenda mejor y más rápido.

fit_transform calcula la escala y transforma el conjunto de entrenamiento.

transform aplica la misma escala al conjunto de prueba.

In [ ]:
reshaped_tensor = tf.reshape(X_train[0], shape=(8, 8))
# Redondeamos para tener una mejor visualizacion
reshaped_tensor =  tf.floor(reshaped_tensor * 100) / 100
print(reshaped_tensor)

tf.reshape(X_train[0], (8, 8)) vuelve a darle forma al primer dato para mostrarlo como imagen de 8x8.


tf.floor(reshaped_tensor * 100) / 100 redondea los valores a solo 2 decimales para que se vea más claro al imprimir.


print(reshaped_tensor) lo muestra en pantalla.



In [ ]:
# Redimensionar el tensor a la forma (total, 8, 8)
X_train = tf.reshape(X_train, (X_train.shape[0], 8, 8))
X_test = tf.reshape(X_test, (X_test.shape[0], 8, 8))

Estamos reconstruyendo las imágenes para que tengan forma de imagen (8x8) en lugar de estar aplastadas en un vector.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D

# Crear el modelo de CNN
model = Sequential([
    Conv2D(32, kernel_size=(3, 3),
        activation='relu',
        input_shape=(8, 8, 1)
    ),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

ste modelo usa una red neuronal convolucional (CNN) para reconocer dígitos convirtiendo la imagen en características y luego clasificándola en una de las 10 clases.

In [ ]:
from tensorflow.keras.optimizers import Adam

# Tasa de aprendizaje deseada
learning_rate = 0.001
adam_optimizer = Adam(learning_rate=learning_rate)

estamos creando el optimizador Adam con una velocidad de aprendizaje de 0.001 para entrenar el modelo de forma estable.

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

Aquí configuramos cómo el modelo va a aprender, cómo va a calcular su error y qué medida de desempeño queremos ver.

In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_split=0.2
)

entrena el modelo usando el 80% de los datos y usa el 20% restante para evaluar cómo va aprendiendo.

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Pérdida de entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida de validación')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()
plt.title('Función de pérdida durante el entrenamiento')
plt.show()


Te permite ver si el modelo mejora o si se está sobreajustando (overfitting).

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

te dice qué tan bien funciona tu modelo con datos nuevos.

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, recall_score

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
# Convert y_test back to multiclass format
y_test_classes = np.argmax(y_test, axis=1)

conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
sensitivity = recall_score(y_test_classes, y_pred_classes, average=None)

Este código convierte las predicciones a clases, genera la matriz de confusión y calcula qué tan bien el modelo reconoce cada número.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Visualizar la matriz de confusión como una imagen de colores
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

# Mostrar la sensibilidad (recall) para cada clase
print('Sensitivity (Recall) for each class:')
for i in range(10):
    print(f'Class {i}: {sensitivity[i]}')

Aquí se visualiza la matriz de confusión como una imagen y se imprime qué tan bien el modelo identifica cada número.

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

# Cambia por el nombre de tu archivo
ruta = "/content/mi_numero.png"
# Escala de grises ("L")
img = Image.open(ruta).convert("L")

# mostrar la imagen
plt.figure(figsize=(4, 4))
plt.imshow(img, cmap="gray")
plt.title("Imagen original")
plt.axis("off")
plt.show()


Este código carga tu imagen, la pasa a blanco y negro y la muestra.

In [ ]:
import matplotlib.pyplot as plt

# redimensionar a 8x8
img_resized = img.resize((8, 8), Image.Resampling.LANCZOS)

# mostrar imagen
plt.figure(figsize=(3, 3))
plt.imshow(img_resized, cmap="gray")
plt.title("Imagen redimensionada a 8x8")
plt.axis("off")
plt.show()

# opcional, guardamos la imagen si es necesario
img_resized.save("imagen_8x8.png")


Este código toma la imagen original, la reduce a 8x8 para que el modelo pueda clasificarla, la muestra y opcionalmente la guarda.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# reescalar de 0-255 a 0-16 e invertir (fondo negro = 0)
img_array = np.array(img_resized).astype("float32")
img_array = 16 - (img_array / 255 * 16)

# mostrar imagen
plt.imshow(img_array, cmap="gray")
plt.title("Imagen reescalada e invertida (0-16)")
plt.axis("off")
plt.show()


Este código convierte la imagen redimensionada al mismo formato numérico que usa el dataset digits (escala 0-16 y colores invertidos) y la muestra.

In [ ]:
from sklearn.preprocessing import StandardScaler

# normalizar
scaler = StandardScaler()
img_flat = img_array
img_scaled = scaler.fit_transform(img_flat)

# Dar forma (1,8,8) que es el input del modelo
img_tensor = img_scaled.reshape(1, 8, 8)

Este código normaliza la imagen y la acomoda con la forma que requiere la red neuronal para poder clasificarla.

In [ ]:
pred = model.predict(img_tensor)
digit = np.argmax(pred)
print("Predicción:", digit)

El modelo analiza la imagen y muestra qué número cree que es.